In [2]:
import os
import matplotlib.pyplot as plt
import pandas as pd

#Step3: Get data for Tissues
Path_to_rpkm='/storage/resources/datasets/gtex/53844/PhenoGenotypeFiles/RootStudyConsentSet_phs000424.GTEx.v6.p1.c1.GRU/ExpressionFiles/phe000006.v2.GTEx_RNAseq.expression-data-matrixfmt.c1/GTEx_Data_20150112_RNAseq_RNASeQCv1.1.8_gene_rpkm.gct.gz'
  #Open to table n skip the first 2 lines
Rpkms = pd.read_csv(Path_to_rpkm, compression='gzip', header=0, sep='\t',skiprows=2, quotechar='"')
#Rpkms.set_index('Name')
Expr_columns=Rpkms.columns
#------------------------------------------------------------------------To be continued

#Step1: Retrieve all Sample Ids from the rpkm file
    #In Code
    #files
exp = open('head_wgs','r')
exp = exp.readlines()
exp = [x.strip('\n')for x in exp]

#Let's Check out these IDs

wgs_table = pd.read_csv('~/projects/GTEX_eSTRs/data/Sra_table_WGS_only', sep='\t', header=None, index_col=0)
Ids = wgs_table[16]
Ids = [x[:9] for x in Ids.tolist() ]
print(len(wgs_table[16]), '\t', len(Ids))

compar = [x for x in exp if x[:9] not in Ids]
#print (compar, Ids)
#-------------------------------------------------------------------IDs checked

#Step2: Get Ids for Tissues
    #In Code
files = os.listdir('/home/szfeupe/projects/GTEX_eSTRs/data/RNA-Seq/')
#print (' '.join(files))


    ##Get RNA IDs by Tissue types
rna = pd.read_csv('~/projects/GTEX_eSTRs/data/Sra_table_RNA-Seq_only', sep='\t', header=None)
rna_id = rna[rna.columns[18:20]]
rna_id.columns=['IDs', 'Tissue']
#pd.value_counts(rna[19].unique())
uniqID = list(set(rna_id.Tissue))      #Unique Tissue names
Count_by_tissue=pd.value_counts(rna[19].values, sort=False)
#print(Count_by_tissue)

  #Group IDs by unique Tissue type
group_id = rna_id['IDs'].groupby(rna_id['Tissue'])
#print(group_id.groups)
#list(group_id.get_group('Whole Blood'))

  #Dictionary of IDs by tissue types
IDbyTissue={x:list(group_id.get_group(x)) for x in uniqID}
#print(IDbyTissue['Kidney - Cortex'])
print('\n\n')


    ##Get IDs for our samples
    ##Use list(set()) to remove duplicate IDs
Samples_IDs={x:list(set([i for i in IDbyTissue[x] if i[:9] in Ids]))  for x in uniqID}

  #There are still duplicated expression for samples: eg. GTEX-N7MS-0007-SM-26GME, GTEX-N7MS-0007-SM-26GMV & GTEX-N7MS-0007-SM-2D7W1
    #Temporarily, We pick the first occurence ( some info -> https://gtexportal.org/home/documentationPage#faqDiv  )
'''
Key=Samples_IDs.keys()
for i in Key:
    dup=[]
    Val=[]
    for V in Samples_IDs[i]:
        if V[:14] not in dup:
            Val.append(V[:23])
            dup.append(V[:14])
    
    Samples_IDs[i]=Val
'''    
for i in Samples_IDs.keys():
    if len(Samples_IDs[i])>=50:
        print(i,'\t', len(Samples_IDs[i]))  #Print Tissues with 50 or more samples

#print('\n','\n'.join(sorted(Samples_IDs['Whole Blood'])))
#-------------------------------------------------------------------IDs by Tissues separated


148 	 148



Skin - Sun Exposed (Lower leg) 	 113
Esophagus - Muscularis 	 102
Esophagus - Mucosa 	 110
Stomach 	 71
Breast - Mammary Tissue 	 51
Artery - Tibial 	 112
Nerve - Tibial 	 97
Cells - Transformed fibroblasts 	 128
Whole Blood 	 165
Testis 	 72
Colon - Transverse 	 65
Artery - Aorta 	 79
Muscle - Skeletal 	 140
Lung 	 145
Cells - EBV-transformed lymphocytes 	 56
Thyroid 	 113
Pancreas 	 68
Heart - Left Ventricle 	 97
Adipose - Subcutaneous 	 114
Adrenal Gland 	 50


/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (25,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def check(X,Y):
    Key=Y
    for i in Key:
        dup=[]
        Val=[]
        for V in X:
            if V[:14] not in dup:
                Val.append(V.strip())
                dup.append(V[:14])
    return Val

print(len(Expr_columns))
Key = Samples_IDs.keys()
f=open('../Expression_data/OutIDs.out', 'w')
#print(Rpkms.columns)
for tissu in Key:
    filename='../Expression_data/'+''.join(tissu.split())
    L = sorted([x for x in Samples_IDs[tissu]])
    
    S=[x for x in L if x in Expr_columns]
    Out=[x for x in L if x not in Expr_columns]
    
    samples=check(S, Samples_IDs.keys())
    if len(samples)>=95:
        print(len(samples),'\t', ''.join(tissu.split()))     #Print tissues with 95 or more samples
#   print(len(samples),' vs ', len(Out), '  samples ',tissu)
    samples.insert(0,'Name')
    
    rpk = Rpkms[samples]
    rpk.to_csv(filename, sep=' ', index=False)
    f.write(tissu+' '+str(len(samples))+'\n'+'\n'.join(Out)+'\n')
    #print (tissu,'...')
    
f.close()
print('All Done')


9592
99 	 Skin-SunExposed(Lowerleg)
102 	 Esophagus-Mucosa
108 	 Artery-Tibial
128 	 Cells-Transformedfibroblasts
144 	 WholeBlood
125 	 Muscle-Skeletal
110 	 Lung
95 	 Thyroid
102 	 Adipose-Subcutaneous
All Done


In [4]:
#get all IDs from sra with RNAseq where all first 9 char match IDs in Ids up there
#How many IDs in total?
#Along with that grab the corresponding tissues so it becomes easier to make a selection later
#Ater selection by tissues, How many?
#Now we go get the actual expression values by IDs compare with previous
Results Before applying the check function (To remove duplicates)

9592
28  vs  4   samples  Brain - Frontal Cortex (BA9)
35  vs  3   samples  Ovary
26  vs  0   samples  Brain - Hypothalamus
29  vs  2   samples  Brain - Nucleus accumbens (basal ganglia)
96  vs  6   samples  Esophagus - Muscularis
27  vs  1   samples  Vagina
5  vs  0   samples  Cervix - Endocervix
7  vs  2   samples  Kidney - Cortex
22  vs  1   samples  Brain - Anterior cingulate cortex (BA24)
156  vs  9   samples  Whole Blood
26  vs  4   samples  Brain - Cerebellar Hemisphere
38  vs  2   samples  Prostate
38  vs  1   samples  Artery - Coronary
51  vs  5   samples  Cells - EBV-transformed lymphocytes
6  vs  0   samples  Fallopian Tube
97  vs  0   samples  Nerve - Tibial
128  vs  17   samples  Lung
78  vs  1   samples  Artery - Aorta
22  vs  3   samples  Pituitary
65  vs  7   samples  Testis
110  vs  4   samples  Adipose - Subcutaneous
0  vs  0   samples  Cells - Leukemia cell line (CML)
29  vs  1   samples  Skin - Not Sun Exposed (Suprapubic)
34  vs  6   samples  Liver
20  vs  2   samples  Brain - Putamen (basal ganglia)
91  vs  6   samples  Heart - Left Ventricle
28  vs  1   samples  Brain - Hippocampus
108  vs  5   samples  Skin - Sun Exposed (Lower leg)
7  vs  2   samples  Bladder
10  vs  1   samples  Minor Salivary Gland
109  vs  3   samples  Artery - Tibial
32  vs  2   samples  Uterus
38  vs  3   samples  Spleen
20  vs  1   samples  Brain - Substantia nigra
17  vs  0   samples  Brain - Spinal cord (cervical c-1)
60  vs  8   samples  Pancreas
29  vs  1   samples  Heart - Atrial Appendage
66  vs  5   samples  Stomach
103  vs  7   samples  Esophagus - Mucosa
27  vs  0   samples  Adipose - Visceral (Omentum)
49  vs  2   samples  Breast - Mammary Tissue
21  vs  2   samples  Brain - Amygdala
26  vs  0   samples  Esophagus - Gastroesophageal Junction
24  vs  5   samples  Brain - Cortex
128  vs  0   samples  Cells - Transformed fibroblasts
32  vs  1   samples  Brain - Caudate (basal ganglia)
27  vs  8   samples  Brain - Cerebellum
133  vs  7   samples  Muscle - Skeletal
18  vs  0   samples  Colon - Sigmoid
12  vs  1   samples  Small Intestine - Terminal Ileum
49  vs  1   samples  Adrenal Gland
6  vs  0   samples  Cervix - Ectocervix
60  vs  5   samples  Colon - Transverse
103  vs  10   samples  Thyroid
All Done+